In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn .metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D , Dense, MaxPool2D
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

R:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
messages = pd.read_csv(r"C:\Users\rhuta\Downloads\spam1.csv", encoding = 'cp1252')

In [3]:
messages

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
6771,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
6772,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
6773,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
6774,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
messages.isnull().sum()

v1               0
v2               0
Unnamed: 2    6720
Unnamed: 3    6760
Unnamed: 4    6768
dtype: int64

In [5]:
messages.rename(columns = {'v1' : 'Label' , 'v2' : 'Message'}, inplace = True)

In [6]:
messages.Label.value_counts()

ham     5854
spam     922
Name: Label, dtype: int64

In [7]:
messages.Label.replace({'spam' :1 , 'ham':0} , inplace = True)

In [8]:
messages.Label.value_counts()

0    5854
1     922
Name: Label, dtype: int64

In [9]:
messages.Message = messages.Message.str.lower()

In [10]:
messages

,Label,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"go until jurong point, crazy.. available only ...",NaN,NaN,NaN
1,0,ok lar... joking wif u oni...,NaN,NaN,NaN
2,1,free entry in 2 a wkly comp to win fa cup fina...,NaN,NaN,NaN
3,0,u dun say so early hor... u c already then say...,NaN,NaN,NaN
4,0,"nah i don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
6771,1,this is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
6772,0,will ì_ b going to esplanade fr home?,NaN,NaN,NaN
6773,0,"pity, * was in mood for that. so...any other s...",NaN,NaN,NaN
6774,0,the guy did some bitching but i acted like i'd...,NaN,NaN,NaN


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
messages_train , messages_test = train_test_split(messages, test_size = 0.2)

In [13]:
messages_train_x = messages_train.iloc[:,1]
messages_train_y = messages_train.iloc[:, 0]

In [14]:
messages_test_x = messages_test.iloc[:,1]
messages_test_y = messages_test.iloc[:, 0]

In [15]:
messages_train_x.shape

(5420,)

In [16]:
messages_train_y = to_categorical(messages_train_y)

#here we have converted our data in 2-dimensional

In [17]:
messages_train_y.shape

(5420, 2)

In [18]:
max_num_words = 12000
#how many unique words we want to consider from the entire corpus

seq_len = 50

#It's nothing but data will be passed in batches of 50 words

embedding_size = 100

#for each word how many embeddings we are going to generate

#All these 3 are hyperparamters

In [19]:
!pip install keras-preprocessing

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer #(It's used to for assigning a number to each unique word)
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
tokenizer = Tokenizer(num_words = max_num_words)

In [22]:
tokenizer.fit_on_texts(messages.Message)
messages_train_x = tokenizer.texts_to_sequences(messages_train_x)
messages_train_x = pad_sequences(messages_train_x, maxlen = seq_len)

In [23]:
messages_test_x = tokenizer.texts_to_sequences(messages_test_x)
messages_test_x = pad_sequences(messages_test_x, maxlen = seq_len)

In [24]:
messages_train_x.shape

(5420, 50)

In [25]:
model = Sequential()

In [26]:
model.add(Embedding(input_dim = max_num_words, input_length = seq_len, output_dim = embedding_size))

R:\Anaconda\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.add(LSTM(2))
model.add(Dense(2, activation='softmax'))
adam = Adam(learning_rate = 0.001)
model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [28]:
model.fit(messages_train_x , messages_train_y, epochs=8 , validation_split = 0.2)

Epoch 1/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8345 - loss: 0.5963 - val_accuracy: 0.9253 - val_loss: 0.3662
Epoch 2/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9532 - loss: 0.3112 - val_accuracy: 0.9686 - val_loss: 0.2153
Epoch 3/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9842 - loss: 0.1747 - val_accuracy: 0.9825 - val_loss: 0.1417
Epoch 4/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9907 - loss: 0.1064 - val_accuracy: 0.9871 - val_loss: 0.1062
Epoch 5/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9979 - loss: 0.0689 - val_accuracy: 0.9889 - val_loss: 0.0860
Epoch 6/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9975 - loss: 0.0508 - val_accuracy: 0.9852 - val_loss: 0.0767
Epoch 7/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9979 - loss: 0.0378 - val_accuracy: 0.9880 - val_loss: 0.0660
Epoch 8/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9992 - loss: 0.0298 - val_accuracy: 0.

In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 100)             │       1,200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 2)                   │             824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,602,492 (13.74 MB)

 Trainable params: 1,200,830 (4.58 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,401,662 (9.16 MB)

In [30]:
pred_values = model.predict(messages_test_x)

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [31]:
pred_values

array([[0.9246338 , 0.07536619],
       [0.9811321 , 0.0188679 ],
       [0.98094004, 0.01905992],
       ...,
       [0.9597949 , 0.04020515],
       [0.9821132 , 0.01788676],
       [0.05533742, 0.94466263]], dtype=float32)

In [33]:
pred_in_classes = np.argmax(pred_values , axis =1)

In [34]:
pred_in_classes

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [43]:
confusion_matrix(messages_test_y , pred_in_classes)

array([[1160,    1],
       [   9,  186]], dtype=int64)

In [44]:
accuracy_score(messages_test_y , pred_in_classes)*100

99.26253687315634